In [ ]:
from pynq import DefaultHierarchy
from pynq import allocate
from math import log
import numpy as np
class FFT_Block_Driver(DefaultHierarchy):
    def __init__(self, description):
        super().__init__(description)
        self.configuration = 0
        self.fft_size = 0

    def _convert_to_data(self,self,fft_direction,size):
        fft_direction.zfill(8)
        byte2 = '0'*8
        x = int(log(size,2))
        fft_size = bin(x)[2:]
        fft_size.zfill(8)
        tdata = fft_direction+byte2+fft_size
        return int(tdata,2)

    def configure(self,fft_direction,fft_size):
        self.configuration = self._convert_to_data(fft_direction,fft_size)
        temp = allocate(1,np.uint32)
        temp[0] = self.configuration
        self.config_dma.sendchannel.transfer(temp)
        self.config_dma.sendchannel.wait()
        del temp

    def stream_fft(self,input_buffer):
        out_buffer = allocate(SAMPLES,np.csingle)
        self.data_dma.sendchannel.transfer(input_buffer)
        self.data_dma.recvchannel.transfer(out_buffer)
        self.data_dma.sendchannel.wait()
        self.data_dma.recvchannel.wait()

        return out_buffer

    @staticmethod
    def check_hierarchy(description):
        if 'data_dma' in description['ip'] \
            and 'config_dma' in desciption['ip']:
            return True
        return False